## Projet de MODS206 - Nowcasting and Dynamic Factor Models

### BASTIDE Simon, RISTORCELLI Ulysse, VUJASINOVIC Thomas

Analyse des bases de données de base, en se restreignant aux fréquences mensuelles :

In [33]:
#imports
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
import plotly.graph_objects as go
import plotly.io as pio
import datetime as dt
pio.templates.default = "plotly_white"

In [34]:
#imports de capitalisations boursières des grandes entreprises du tourisme
def get_data(assets, start_date, end_date):
    """
    Récupère les données des actions sur Yahoo Finance
    """
    return yf.download(assets, start=start_date, end=end_date)['Adj Close']

assets = ['0H59.IL', 'ALVDM.PA']
N = 5000 # Complétez avec le nombre de jours historiques

start_date = dt.datetime.now() - dt.timedelta(days=N)
end_date = dt.datetime.now()

data = get_data(assets, start_date, end_date)
data
monthly_data = data.resample('M').mean()
monthly_data


[*********************100%%**********************]  2 of 2 completed


,0H59.IL,ALVDM.PA
Date,,
2010-07-31,19.698264,15.172885
2010-08-31,19.460198,15.454809
2010-09-30,20.874527,15.673765
2010-10-31,22.767187,17.284562
2010-11-30,24.577961,18.513290
...,...,...
2023-11-30,30.999091,112.863636
2023-12-31,34.373947,120.447368
2024-01-31,35.514091,124.954545


In [35]:
# Affichage des données superposé avec plotly pour une année 
fig = go.Figure()
for a in assets:
    fig.add_trace(go.Scatter(x=data.index, y=data[a], name=a))
fig.update_layout(title='Evolution des prix des actifs', yaxis_title='Prix en $')
fig.show()

In [36]:
sight_seeing_demand_df=pd.read_csv('databases/famille_SUIVI-DEMANDE-TOURISTIQUE/valeurs_mensuelles.csv', sep=';')
sight_seeing_demand_df=sight_seeing_demand_df.drop(sight_seeing_demand_df[sight_seeing_demand_df['Libellé'] == 'Codes'].index)
sight_seeing_demand_df=sight_seeing_demand_df.drop('Période', axis=1)
sight_seeing_demand_df

overnight_stays_df=pd.read_csv('databases/famille_NUITEE/valeurs_mensuelles.csv', sep=';')
overnight_stays_df=overnight_stays_df.drop(overnight_stays_df[overnight_stays_df['Libellé'] == 'Codes'].index)
overnight_stays_df=overnight_stays_df.drop('Période',axis=1)

regions=[
    "Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne",
    "Centre-Val de Loire", "Corse", "Grand Est", "Hauts-de-France",
    "Ile-de-France", "Normandie", "Nouvelle-Aquitaine", "Occitanie",
    "Pays de la Loire", "Provence Alpes Côte d’Azur"
]

overnight_stays_df_t = overnight_stays_df.transpose()
# Rename columns using the first row, cleansing of not useful columns
overnight_stays_df_t.columns = overnight_stays_df_t.iloc[0]
overnight_stays_df_t = overnight_stays_df_t.drop(overnight_stays_df_t.index[:3])

#overnight_stays_df_t = overnight_stays_df_t.iloc[:, list(range(3, 21)) + list(range(64, 80))]
occupation_columns = [col for col in overnight_stays_df_t.columns if "Taux d'occupation dans l'hôtellerie" in col]
overnight_columns = [col for col in overnight_stays_df_t.columns if "Nuitées dans l'hôtellerie - Total -" in col]
arrival_columns= [col for col in overnight_stays_df_t.columns if "Arrivées dans l'hôtellerie - Total -" in col]

selected_occupation_columns = [col for col in occupation_columns if any(region in col for region in regions)]
selected_overnight_columns = [col for col in overnight_columns if any(region in col for region in regions)]
selected_arrival_columns = [col for col in arrival_columns if any(region in col for region in regions)]

selected_columns = selected_arrival_columns+selected_occupation_columns+selected_overnight_columns
overnight_stays_df_t =overnight_stays_df_t[selected_columns]
#overnight_stays_df_t["Taux"]
overnight_stays_df_t

Libellé,Arrivées dans l'hôtellerie - Total - Centre-Val de Loire - Série arrêtée,Arrivées dans l'hôtellerie - Total - Bourgogne-Franche-Comté - Série arrêtée,Arrivées dans l'hôtellerie - Total - Normandie - Série arrêtée,Arrivées dans l'hôtellerie - Total - Hauts-de-France - Série arrêtée,Arrivées dans l'hôtellerie - Total - Grand Est - Série arrêtée,Arrivées dans l'hôtellerie - Total - Pays de la Loire - Série arrêtée,Arrivées dans l'hôtellerie - Total - Bretagne - Série arrêtée,Arrivées dans l'hôtellerie - Total - Nouvelle-Aquitaine - Série arrêtée,Arrivées dans l'hôtellerie - Total - Occitanie - Série arrêtée,Arrivées dans l'hôtellerie - Total - Auvergne-Rhône-Alpes - Série arrêtée,...,Nuitées dans l'hôtellerie - Total - Normandie,Nuitées dans l'hôtellerie - Total - Auvergne-Rhône-Alpes,Nuitées dans l'hôtellerie - Total - Grand Est,Nuitées dans l'hôtellerie - Total - Nouvelle-Aquitaine,Nuitées dans l'hôtellerie - Total - Occitanie,Nuitées dans l'hôtellerie - Total - Corse-du-Sud,Nuitées dans l'hôtellerie - Total - Bourgogne-Franche-Comté,Nuitées dans l'hôtellerie - Total - Haute-Corse,Nuitées dans l'hôtellerie - Total - Centre-Val de Loire,Nuitées dans l'hôtellerie - Total - Hauts-de-France
2010-01,166.87,241.91,210.05,308.91,389.87,207.7,170.45,379.49,389.47,793.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02,189.11,308.87,252.95,338.58,445.58,218.23,203.79,457.31,461.73,918.92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03,249.37,333.78,318.79,403.19,522.0,287.74,264.82,567.07,542.73,971.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04,315.19,414.1,430.0,476.24,627.02,312.64,344.17,730.62,753.26,932.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05,385.39,474.67,478.52,517.69,731.28,347.41,398.96,793.95,898.14,965.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,838,2053,1419,1654,1691,274.56,712,206.43,613,886
2023-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,674,1650,1238,1369,1362,141.22,581,87.56,511,805
2023-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,470,1330,1005,913,838,26.35,427,23.84,369,662
2023-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,421,1899,1357,891,834,20.76,427,19.56,353,670


In [68]:
change_rate_pd=pd.read_csv('databases/euro change rates.csv',sep=',')

change_rate_pd['Date'] = pd.to_datetime(change_rate_pd['Date'])

change_rate_pd=change_rate_pd[change_rate_pd['Date']>"2010-12-31"]
change_rate_pd.drop(columns=['Unnamed: 42'], inplace=True)

change_rate_pd = change_rate_pd.resample('M', on='Date').mean()

change_rate_pd

,USD,JPY,BGN,CYP,CZK,DKK,EEK,GBP,HUF,LTL,...,ILS,INR,KRW,MXN,MYR,NZD,PHP,SGD,THB,ZAR
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-31,1.335967,110.380952,1.9558,NaN,24.448810,7.451838,NaN,0.847117,275.326190,3.4528,...,4.790919,60.716086,1495.502857,16.192614,4.089490,1.743543,59.088952,1.719300,40.826571,9.265152
2011-02-28,1.364895,112.768000,1.9558,NaN,24.276550,7.455480,NaN,0.846355,271.152500,3.4528,...,4.993850,62.014225,1524.990500,16.472735,4.154105,1.792520,59.557600,1.742120,41.917500,9.812615
2011-03-31,1.399917,114.397391,1.9558,NaN,24.393000,7.457430,NaN,0.866533,270.890435,3.4528,...,4.986696,62.952635,1568.054348,16.806339,4.248304,1.887739,60.870391,1.775735,42.506391,9.686152
2011-04-30,1.444179,120.415263,1.9558,NaN,24.300895,7.457405,NaN,0.882913,265.287368,3.4528,...,4.957332,64.112789,1567.515789,16.921132,4.350226,1.833079,62.361211,1.802416,43.434474,9.719958
2011-05-31,1.434864,116.467273,1.9558,NaN,24.381318,7.456614,NaN,0.877877,266.958182,3.4528,...,4.974045,64.473455,1555.989545,16.717650,4.327218,1.802441,61.952773,1.776250,43.397955,9.846141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,1.080823,161.844091,1.9558,NaN,24.484773,7.458086,NaN,0.870454,379.194545,NaN,...,4.111768,90.010200,1412.571818,18.787691,5.066686,1.803295,60.284591,1.457527,38.327364,20.039686
2023-12-31,1.090305,157.212632,1.9558,NaN,24.477579,7.455637,NaN,0.861683,381.803158,NaN,...,4.005658,90.800937,1423.126316,18.766526,5.082553,1.755584,60.593000,1.453263,38.175579,20.337326
2024-01-31,1.090514,159.458182,1.9558,NaN,24.715818,7.457200,NaN,0.858731,382.041818,NaN,...,4.049314,90.634745,1447.184091,18.626832,5.110464,1.767264,61.089636,1.456873,38.400682,20.516095


In [ ]:
cpi_index_df=pd.read_csv("databases/CPI deflated EER-41Euro.csv",sep=',')
pd.to_datetime(cpi_index_df['DATE'])
cpi_index_df=cpi_index_df[cpi_index_df['DATE']>"2010-12-31"]
cpi_index_df


,DATE,TIME PERIOD,CPI deflated EER-41/Euro (EXR.M.E03.EUR.ERC0.A)
216,2011-01-31,2011Jan,96.7008
217,2011-02-28,2011Feb,97.8516
218,2011-03-31,2011Mar,99.3829
219,2011-04-30,2011Apr,100.9849
220,2011-05-31,2011May,99.9160
...,...,...,...
369,2023-10-31,2023Oct,95.0032
370,2023-11-30,2023Nov,95.3326
371,2023-12-31,2023Dec,94.8389
372,2024-01-31,2024Jan,95.1396
